In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm, trange
from math import sqrt, inf
import pandas as pd
import numpy as np
import random
import math

In [2]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [3]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
y_ui = np.zeros((n, m))
for i in base.itertuples():
    if i[3]>=4:
        train_matrix[i[1] - 1, i[2] - 1] = 1

for i in test.itertuples():
    if i[3]>=4:
        test_matrix[i[1] - 1, i[2] - 1] = 1
# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i,j] >= 4:
            y_ui[i,j] = 1
# 确定物品集与用户集
I = list(np.where(sum(train_matrix+test_matrix)!=0)[0])
U = list(np.where(sum(train_matrix.T+test_matrix.T)!=0)[0])
# 求 I|Iu 
Iu = {}
for u in U:
    Iu.setdefault(u,[])
    Iu[u] = list(set(np.where(train_matrix[u]!=0)[0]))
# 观测到的（用户，物品）对的完整集合
P = list()
for u in U:
    ls = list(set(np.where(train_matrix[u]!=0)[0]))
    for i in ls:
        P.append((u,i))
P = np.array(P)
# 未观测到的（用户，物品）对的集合
A = list()
for u in U:
    ls = list(set(I)-set(np.where(train_matrix[u]!=0)[0]))
    for i in ls:
        A.append((u,i))
A = np.array(A)

In [4]:
# evaluate function
def PreK(rank, truth, k):
    prek = 0
    ure = 0
    for i in U:
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        preuk = 0
        ure+=1
        for j in range(k):
            if rank[i][j] in ls:
                preuk+=1
        prek+=preuk/k
    return prek/ure
def RecK(rank, truth, k):
    reck = 0
    ure = 0
    for i in U:
        ls = list(np.where(truth[i]!=0)[0])
        ts = len(ls)
        if ts==0:
            continue
        recuk = 0
        ure += 1
        for j in range(k):
            if rank[i][j] in ls:
                recuk += 1
        reck+=recuk/ts
    return reck/ure

In [5]:
def FISM(R, P, A, Uuk, Wik, Vik, bik, buk, T, alpha, alpha_v, alpha_w, beta_u, beta_v, gama):
    num = len(P)
    for _ in tqdm(range(T)):
        # Randomly pick up a set A with |A| = ρ|P|
        Ak = np.random.permutation(A)
        idx = num*3
        Au = Ak[0:idx]
        Pua = np.random.permutation(np.append(P,Au, axis=0))
        for k in Pua:
            # Calculate Uuk
            u,i = k
            tmp = np.array([0.0]*d)
            q = 0
            Iu_copy = Iu[u].copy()
            for j in Iu[u]:
                if j!=i:
                    tmp += Wik[j]
                    q+=1
                else:
                    Iu_copy.remove(j)
            if q==0:
                continue
            tmp /= np.power(q,alpha)
            Uuk[u] = tmp
            # Calculate rui
            rui = np.dot(Uuk[u],Vik[i].T)+buk[u]+bik[i]
            eui = R[u,i]-rui
            # Calculate the gradient of each parameters
            grad_Vi = -eui*Uuk[u]+alpha_v*Vik[i]
            grad_Wi = -eui*(1.0/np.power(q,alpha))*Vik[i]+alpha_w*Wik[Iu_copy]
            grad_bu = -eui*beta_u*buk[u]
            grad_bi = -eui*beta_v*bik[i]
            # update the model parameters
            Vik[i] = Vik[i]-gama*grad_Vi
            Wik[Iu_copy] = Wik[Iu_copy]-gama*grad_Wi
            buk[u] = buk[u]-gama*grad_bu
            bik[i] = bik[i]-gama*grad_bi
    return Uuk, Vik, buk, bik

In [8]:
if __name__ == '__main__':
    gamma = 0.01
    d = 20
    alpha = 0.5
    rho = 3
    alpha_v = alpha_w = beta_u = beta_v = 0.001
    T = 100
    avg = np.sum(y_ui)/n/m
    bi = np.sum(y_ui, axis=0)/n - avg
    bu = np.sum(y_ui, axis=1)/m - avg
    Wi = (np.random.rand(m+1, d) - 0.5) * 0.01
    Vi = (np.random.rand(m+1, d) - 0.5) * 0.01
    Uu = (np.random.rand(n+1, d) - 0.5) * 0.01
    Uu, Vi, bu, bi = FISM(train_matrix, P, A, Uu, Wi, Vi, bi, bu, T, alpha, alpha_v, alpha_w, beta_u, beta_v, gamma)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [46:11<00:00, 27.71s/it]


In [9]:
fism = np.zeros((n, m))
rank_fism = list()
for u in U:
    for i in I:
        fism[u,i] = np.dot(Uu[u],Vi[i].T)+bu[u]+bi[i]
for i in tqdm(U):
    exclude = list(np.where(train_matrix[i,:]!=0)[0])
    ls = list()
    for j in I:
        if j in exclude:
            continue
        ls.append((j,fism[i,j]))
    ls = sorted(ls, key=lambda x:x[1], reverse=True)
    rank_fism.append([ls[j][0] for j in range(len(ls))])

100%|███████████████████████████████████████████████████████████████████████████████| 942/942 [00:02<00:00, 373.42it/s]


In [10]:
print("FISM Pre@5:"+str(PreK(rank_fism, test_matrix, 5)))
print("FISM Rec@5:"+str(RecK(rank_fism, test_matrix, 5)))

FISM Pre@5:0.38991228070175427
FISM Rec@5:0.1217583619171398
